# the 1st step is to export libraries

In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


# the 2nd step is to define connection details

In [2]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=""   #REPLACE THIS WITH THE PASSWORD YOU SET
)
 
print(mydb)
 
if mydb.is_connected():
    print("CONNECTION SUCCESSFUL")


CONNECTION SUCCESSFUL


# the 3rd step is to connect to the MySQL server

In [3]:
mycursor = mydb.cursor()
    
mycursor.execute("CREATE DATABASE medical_practice")

# the 4th step is DB Schema Code 1 - create tables

In [4]:
mycursor.execute("CREATE TABLE medical_practice.patients ("
                     "ssn VARCHAR(128) not null PRIMARY KEY,"
                     "name VARCHAR(255),"
                     "address VARCHAR(255),"
                     "birthdate DATETIME,"
                     "physician_id VARCHAR(128))")

In [5]:
mycursor.execute("CREATE TABLE medical_practice.physicians ("
                     "SSN VARCHAR(128) not null not null PRIMARY KEY,"
                     "name VARCHAR(255),"
                     "primary_specialty VARCHAR(255),"
                     "experience_years INT,"
                     "check (experience_years >= 0))") 

In [6]:
mycursor.execute("CREATE TABLE medical_practice.pharmacies ("
                     "id INT not null PRIMARY KEY,"
                     "name VARCHAR(255),"
                     "address VARCHAR(255),"
                     "phone VARCHAR(128))")

In [7]:
mycursor.execute("CREATE TABLE medical_practice.drugs ("
                     "drug_id INT,"
                     "drug_name VARCHAR(255) not null PRIMARY KEY)")

In [8]:
mycursor.execute("CREATE TABLE medical_practice.prescriptions ("
                     "id INT not null PRIMARY KEY,"
                     "patient_id VARCHAR(128),"
                     "physician_id VARCHAR(128),"
                     "drug_name VARCHAR(255),"
                     "date DATE,"
                     "quantity INT)")

In [9]:
mycursor.execute("CREATE TABLE medical_practice.adverse_reactions ("
                     "drug_name_1 VARCHAR(255) not null,"
                     "drug_name_2 VARCHAR(255) not null,"
                     "PRIMARY KEY (drug_name_1, drug_name_2))")

In [10]:
mycursor.execute("CREATE TABLE medical_practice.alerts ("
                     "patient_id VARCHAR(128),"
                     "physician_id VARCHAR(128),"
                     "alert_date DATE,"
                     "drug1 VARCHAR(255),"
                     "drug2 VARCHAR(255))")

In [11]:
mycursor.execute("CREATE TABLE medical_practice.pharmacy_fills ("
                     "pharmacy_id INT not null ,"
                     "prescription_id INT not null ,"
                     "date DATE,"
                     "cost DECIMAL(10, 2),"
                     "PRIMARY KEY (pharmacy_id, prescription_id))")

In [12]:
mycursor.execute("CREATE TABLE medical_practice.companies ("
                     "id INT not null PRIMARY KEY,"
                     "name VARCHAR(255),"
                     "address VARCHAR(255),"
                     "contact_phone VARCHAR(128),"
                     "contact_name VARCHAR(255))")

In [13]:
mycursor.execute("CREATE TABLE medical_practice.contracts ("
                     "Contract_Id INT not null PRIMARY KEY,"
                     "drug VARCHAR(255),"
                     "dosage INT,"
                     "pharmacy_id INT,"
                     "pharm_company_id INT,"
                     "quantity INT,"
                     "date DATE,"
                     "price INT check (price >= 0))")

In [14]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="", #REPLACE THIS WITH YOUR PASSWORD 
  database = "medical_practice" 
)

In [15]:
mycursor = mydb.cursor()

In [16]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
      print(x)

('adverse_reactions',)
('alerts',)
('companies',)
('contracts',)
('drugs',)
('patients',)
('pharmacies',)
('pharmacy_fills',)
('physicians',)
('prescriptions',)


# the 5th step is DB Schema Code 2 - define foreign keys

In [17]:
query = ('ALTER TABLE patients ADD FOREIGN KEY (physician_id) REFERENCES physicians(SSN)')
mycursor.execute(query)

In [18]:
query = ('ALTER TABLE prescriptions ADD FOREIGN KEY (patient_id) REFERENCES patients(ssn)')
mycursor.execute(query)

In [19]:
query = ('ALTER TABLE prescriptions ADD FOREIGN KEY (physician_id) REFERENCES physicians(SSN)')
mycursor.execute(query)

In [20]:
mycursor.execute("CREATE UNIQUE INDEX idx_drug_name ON drugs(drug_name)")

In [21]:
query = ('ALTER TABLE prescriptions ADD FOREIGN KEY (drug_name) REFERENCES drugs(drug_name)')
mycursor.execute(query)

In [22]:
query = ('ALTER TABLE adverse_reactions ADD FOREIGN KEY (drug_name_1) REFERENCES drugs(drug_name)')
mycursor.execute(query)

In [23]:
query = ('ALTER TABLE adverse_reactions ADD FOREIGN KEY (drug_name_2) REFERENCES drugs(drug_name)')
mycursor.execute(query)

In [24]:
query = ('ALTER TABLE alerts ADD FOREIGN KEY (patient_id) REFERENCES patients(ssn)')
mycursor.execute(query)

In [25]:
query = ('ALTER TABLE alerts ADD FOREIGN KEY (physician_id) REFERENCES physicians(SSN)')
mycursor.execute(query)

In [26]:
mycursor.execute("CREATE INDEX idx_prescriptions ON prescriptions(patient_id,drug_name)")

In [27]:
query = ('ALTER TABLE alerts ADD FOREIGN KEY (patient_id,drug1) REFERENCES prescriptions(patient_id,drug_name)')
mycursor.execute(query)

In [28]:
query = ('ALTER TABLE alerts ADD FOREIGN KEY (patient_id,drug2) REFERENCES prescriptions(patient_id,drug_name)')
mycursor.execute(query)

In [29]:
query = ('ALTER TABLE pharmacy_fills ADD FOREIGN KEY (prescription_id) REFERENCES prescriptions(id)')
mycursor.execute(query)

In [30]:
query = ('ALTER TABLE pharmacy_fills ADD FOREIGN KEY (pharmacy_id) REFERENCES pharmacies(id)')
mycursor.execute(query)

In [31]:
query = ('ALTER TABLE contracts ADD FOREIGN KEY (pharm_company_id) REFERENCES companies(id)')
mycursor.execute(query)

In [32]:
query = ('ALTER TABLE contracts ADD FOREIGN KEY (pharmacy_id) REFERENCES pharmacies(id)')
mycursor.execute(query)

In [33]:
query = ('ALTER TABLE contracts ADD FOREIGN KEY (drug) REFERENCES drugs(drug_name)')
mycursor.execute(query)

# the 6th step is Data Insertion Code

In [34]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/drugs.csv")
data.head()

,drug_id,drug_name
0,1,Primalovir
1,2,Olanzanafine
2,3,Avafoxin
3,4,Quixiposide
4,5,Cleotrana


In [35]:
for i,row in data.iterrows():
            sql = "INSERT INTO drugs VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [36]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/pharmacies.csv")
data.head()

,id,name,address,phone
0,1,Springfield Pharmacy,"123 Main St, Springfield, IL 62701",(217) 555-1234
1,2,Peachtree Meds,"456 Elm St, Atlanta, GA 30303",(404) 555-5678
2,3,Lone Star Drugs,"789 Oak St, Dallas, TX 75201",(214) 555-9101
3,4,Mile High Meds,"101 Pine St, Denver, CO80202",(303) 555-1121
4,5,Emerald City Pharmacy,"121 Spruce St, Seattle, WA 98101",(206) 555-1314


In [37]:
for i,row in data.iterrows():
            sql = "INSERT INTO pharmacies VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [38]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/physicians.csv")
data.head()

,SSN,name,primary_specialty,experience_years
0,614-57-6885,Srinivasan,Cardiology,4
1,702-16-8749,Wu,Dermatology,10
2,571-13-9020,Mozart,Cardiology,0
3,718-27-0905,Einstein,Psychiatry,29
4,230-12-3219,El Said,Psychiatry,12


In [39]:
for i,row in data.iterrows():
            sql = "INSERT INTO physicians VALUES (%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [40]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/companies.csv")
data.head()

,id,name,address,contact_phone,contact_name
0,1,Goodrx,"123 Main St, San Francisco, CA",123-456-7890,Holly Jolly
1,2,PHARMASEE,"456 Elm St, New York, NY",234-567-8901,Faker Maker
2,3,DRUGXO,"789 Maple St, Los Angeles, CA",345-678-9012,Silly Putty
3,4,Pharmachoice,"101 Pine St, Chicago, IL",456-789-0123,Connie Honey
4,5,Castox,"234 Oak St, Houston, TX",567-890-1234,Laxmi Kant Sheth


In [41]:
for i,row in data.iterrows():
            sql = "INSERT INTO companies VALUES (%s,%s,%s,%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [42]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/patients.csv")
data.head()

,ssn,name,address,birthdate,physician_id
0,478-34-0781,Florance Saiz,7 Fair Oaks Place,1988-11-03T23:25:38Z,614-57-6885
1,885-94-4721,Merry Di Pietro,1 Old Shore Court,1991-02-07T22:00:41Z,702-16-8749
2,777-39-3296,Myron Cottem,75875 Fulton Crossing,1986-02-20T04:43:29Z,718-27-0905
3,227-08-7452,Bearnard Remer,18669 Heffernan Point,2008-01-09T05:34:30Z,230-12-3219
4,805-15-2755,Roxana Worster,54 Hudson Junction,1982-11-12T18:11:55Z,163-50-5535


In [43]:
for _, row in data.iterrows():
    sql = "INSERT INTO patients (ssn, name, address, birthdate, physician_id) VALUES (%s, %s, %s, %s, %s)"
    
    original_date = row['birthdate']
    date_time = original_date.split('T')
    date_str = date_time[0]
    time_str = date_time[1][:-1]
    
    mycursor.execute(sql, (row['ssn'], row['name'], row['address'], f"{date_str} {time_str}", row['physician_id']))
    print("Record inserted")
    mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [44]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/prescriptions.csv")
data.head()

,id,patient_id,physician_id,drug_name,date,quantity
0,1,478-34-0781,614-57-6885,Avafoxin,3/11/2023,90
1,2,758-08-7274,118-66-5958,Cleotrana,3/12/2023,10
2,3,758-08-7274,118-66-5958,Primalovir,4/11/2023,20
3,4,758-08-7274,118-66-5958,Glucozepam Amcipentin,5/13/2023,12
4,5,303-13-5928,571-13-9020,Olanzanafine,5/24/2023,25


In [45]:
from datetime import datetime

for i, row in data.iterrows():
    
    date_str = datetime.strptime(row['date'], '%m/%d/%Y').strftime('%Y-%m-%d')
    sql = "INSERT INTO prescriptions (id, patient_id, physician_id, drug_name, date, quantity) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['id'], row['patient_id'], row['physician_id'], row['drug_name'], date_str, row['quantity'])
    
    mycursor.execute(sql, values)
    print("Record inserted")
    mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [46]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/pharmacy_fills.csv")
data.head()

,pharmacy_id,prescription_id,date,cost
0,1,3,4/15/2023,60.53
1,3,4,5/16/2023,41.50
2,1,2,3/12/2023,18.00
3,8,1,3/12/2023,146.53
4,10,5,5/26/2023,47.50


In [47]:
for i, row in data.iterrows():
    
    date_str = datetime.strptime(row['date'], '%m/%d/%Y').strftime('%Y-%m-%d')
    sql = "INSERT INTO pharmacy_fills (pharmacy_id, prescription_id, date, cost) VALUES (%s, %s, %s, %s)"
    values = (row['pharmacy_id'], row['prescription_id'], date_str, row['cost'])
    
    mycursor.execute(sql, values)
    print("Record inserted")
    mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [48]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/adverse_reactions.csv")
data.head()

,drug_name_1,drug_name_2
0,Cleotrana,Kanulin
1,Primalovir,Abnazole Toleluble
2,Primalovir,Olanzanafine
3,Olanzanafine,Glucozepam Amcipentin
4,Avafoxin,Kanulin


In [49]:
for i,row in data.iterrows():
            sql = "INSERT INTO adverse_reactions VALUES (%s,%s)"
            mycursor.execute(sql, tuple(row))
            print("Record inserted")
            mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [50]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/alerts.csv")
data.head()

,patient_id,physician_id,alert_date,drug1,drug2
0,303-13-5928,571-13-9020,5/24/2023,Olanzanafine,Primalovir
1,303-13-5928,571-13-9020,5/24/2023,Primalovir,Abnazole Toleluble
2,303-13-5928,571-13-9020,6/22/2023,Olanzanafine,Glucozepam Amcipentin
3,478-34-0781,614-57-6885,9/17/2023,Avafoxin,Quixiposide
4,501-47-2038,156-28-1945,9/22/2023,Cleotrana,Avafoxin


In [51]:
for i, row in data.iterrows():
    
    date_str = datetime.strptime(row['alert_date'], '%m/%d/%Y').strftime('%Y-%m-%d')
    sql = "INSERT INTO alerts (patient_id, physician_id, alert_date, drug1, drug2) VALUES (%s, %s, %s, %s, %s)"
    values = (row['patient_id'], row['physician_id'], date_str, row['drug1'], row['drug2'])
    
    mycursor.execute(sql, values)
    print("Record inserted")
    mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


In [52]:
data = pd.read_csv("/Users/emmasun/Desktop/DEMO/contracts.csv")
data.head()

,Contract_Id,drug,dosage,pharmacy_id,pharm_company_id,quantity,date,price
0,1,Cleotrana,50,5,2,40,10/3/2023,100.0
1,2,Primalovir,500,5,10,20,9/26/2023,40.5
2,3,Kanulin,1000,4,9,20,9/20/2023,10.5
3,4,Olanzanafine,50,6,3,80,9/24/2023,150.0
4,5,Avafoxin,5,15,5,30,9/29/2023,18.0


In [53]:
for i, row in data.iterrows():
    
    date_str = datetime.strptime(row['date'], '%m/%d/%Y').strftime('%Y-%m-%d')
    sql = "INSERT INTO contracts (Contract_Id, drug, dosage, pharmacy_id, pharm_company_id, quantity, date, price) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    values = (row['Contract_Id'], row['drug'],row['dosage'],row['pharmacy_id'],row['pharm_company_id'],row['quantity'], date_str, row['price'])
    
    mycursor.execute(sql, values)
    print("Record inserted")
    mydb.commit()

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted


# the 7th step is DB Schema Code 3 - stored procedure and trigger

In [54]:
# The hospital administration needs to retrieve from time to time primary specialty and years of experience of a given physician.
# Write a stored procedure that can be used to do that.

In [55]:
mycursor.execute("""
CREATE PROCEDURE physician_info(IN info_SSN VARCHAR(128), OUT info_specialty VARCHAR(255), OUT info_years INT)
BEGIN
    SELECT physicians.primary_specialty, physicians.experience_years
    INTO info_specialty, info_years
    FROM physicians
    WHERE physicians.SSN = info_SSN;
END;
""")

In [56]:
# Write a trigger that inserts a tuple in the alerts table when a patient is prescribed a drug
# (i.e., after inserting a new row in the prescriptions table) 
# that adversely interacts with a drug the patient was earlier prescribed. 

In [57]:
mycursor.execute("""
CREATE TRIGGER prescription_alert AFTER INSERT ON prescriptions
FOR EACH ROW
BEGIN
   
    DECLARE alertdrug_name VARCHAR(255);

    SELECT prescriptions.drug_name INTO alertdrug_name
    FROM prescriptions
    JOIN adverse_reactions ON (NEW.drug_name = adverse_reactions.drug_name_1 AND prescriptions.drug_name = adverse_reactions.drug_name_2)
                                OR (NEW.drug_name = adverse_reactions.drug_name_2 AND prescriptions.drug_name = adverse_reactions.drug_name_1)
    WHERE NEW.patient_id = prescriptions.patient_id 
    AND prescriptions.drug_name != NEW.drug_name
    AND prescriptions.date < NEW.date limit 1;

    IF alertdrug_name IS NOT NULL THEN
        INSERT INTO alerts (patient_id, physician_id, alert_date, drug1, drug2)
        VALUES (NEW.patient_id, NEW.physician_id, NEW.date, alertdrug_name, NEW.drug_name);
    END IF;
END;
""")

# the 8th step is to write SQL queries

In [79]:
# 1. Find the physicians (ssn) who have most prescribed drugs which caused alerts 
# (due to possible adverse interaction with a previously prescribed drug, not necessarily by the same physician).

In [80]:
mycursor.execute("""
with alert_count as (select physicians.ssn as physicianssn, count(alerts.patient_id) as totalnumber
                     from physicians 
                     join prescriptions on physicians.ssn = prescriptions.physician_id 
                     join alerts on prescriptions.patient_id = alerts.patient_id
                     group by physicians.ssn)
select alert_count.physicianssn
from alert_count
where alert_count.totalnumber = (select max(totalnumber) from alert_count)
""")

In [81]:
for row in mycursor.fetchall():
    print(row)

('571-13-9020',)
('156-28-1945',)


In [82]:
# 2.Find the physicians (ssn) who have prescribed two drugs to the same patient which have adverse interactions.


In [83]:
mycursor.execute("""
select distinct physicians.SSN
from physicians 
join prescriptions P1 on physicians.SSN = P1.physician_id 
join prescriptions P2 on P1.patient_id = P2.patient_id 
join adverse_reactions on P1.drug_name = adverse_reactions.drug_name_1 and P2.drug_name = adverse_reactions.drug_name_2
where P1.drug_name != P2.drug_name
""")

In [84]:
for row in mycursor.fetchall():
    print(row)

('571-13-9020',)
('156-28-1945',)
('614-57-6885',)


In [85]:
# 3.Find the physicians who have prescribed most drugs supplied by company DRUGXO.

In [86]:
mycursor.execute("""
select physicians.SSN
from physicians 
join prescriptions on physicians.SSN = prescriptions.physician_id 
join contracts on prescriptions.drug_name = contracts.drug 
join companies on contracts.pharm_company_id = companies.id
where companies.name = 'DRUGXO'
group by physicians.SSN
order by count(contracts.Contract_Id) desc limit 1
""")

In [87]:
for row in mycursor.fetchall():
    print(row)

('571-13-9020',)


In [88]:
# 4.For each drug supplied by company PHARMASEE display the price (per unit of quantity) charged by that company for that drug 
# along with the average price charged for that drug (by companies, not pharmacies). 
# Note: As it happens in the data we supplied each drug is supplied by only one company, 
# but your query should not be based on that.

In [89]:
mycursor.execute("""
with PHARMASEE_AVG as (select contracts.drug, SUM(contracts.price) / SUM(contracts.quantity) as price_average
                       from contracts join companies on contracts.pharm_company_id = companies.id
                       where companies.name = 'PHARMASEE'
                       group by contracts.drug)
select (contracts.price / contracts.quantity) as price_per_unit_of_quantity, PHARMASEE_AVG.price_average
from contracts 
join companies on contracts.pharm_company_id = companies.id 
join PHARMASEE_AVG on contracts.drug = PHARMASEE_AVG.drug
where companies.name = 'PHARMASEE'
""")

In [90]:
for row in mycursor.fetchall():
    print(row)

(Decimal('2.5000'), Decimal('2.5000'))
(Decimal('1.8125'), Decimal('2.2500'))
(Decimal('4.0000'), Decimal('2.2500'))


In [91]:
# 5.For each drug and for each pharmacy, 
# find the percentage of the markup (per unit of quantity) for that drug by that pharmacy.

In [92]:
mycursor.execute("""
select contracts.drug, contracts.pharmacy_id,((pharmacy_fills.cost/prescriptions.quantity) - (contracts.price/contracts.quantity)) / (contracts.price/contracts.quantity) * 100 as percentage_of_markup
from pharmacy_fills
join prescriptions on pharmacy_fills.prescription_id = prescriptions.id
right join contracts on prescriptions.drug_name = contracts.drug and contracts.pharmacy_id = pharmacy_fills.pharmacy_id
""")

In [93]:
for row in mycursor.fetchall():
    print(row)

('Cleotrana', 5, None)
('Primalovir', 5, None)
('Kanulin', 4, Decimal('871.5909090909'))
('Olanzanafine', 6, None)
('Avafoxin', 15, None)
('Quixiposide', 5, None)
('Glucozepam Amcipentin', 10, None)
('Divisporine Acetaclotide', 12, None)
('Abnazole Toleluble', 14, None)
('Dantopex Quixilinum', 14, None)
('Olanzanafine', 1, None)
('Olanzanafine', 2, None)
('Olanzanafine', 3, None)
('Glucozepam Amcipentin', 8, None)
('Divisporine Acetaclotide', 9, None)
('Dantopex Quixilinum', 13, None)
('Glucozepam Amcipentin', 4, None)
('Avafoxin', 8, Decimal('171.3518518333'))


In [94]:
# 6.For each drug, find the average time between when a patient was prescribed a drug 
# and when the prescription was filled at a pharmacy. 
# (You will need to extract the components of a date to compute this. 
# SQL provides functions for doing that—please Google for this if I don't cover this in class next week).

In [95]:
mycursor.execute("""
select prescriptions.drug_name, AVG(TIMESTAMPDIFF(DAY, prescriptions.date, pharmacy_fills.date)) AS average_time_between
from prescriptions 
left join pharmacy_fills on prescriptions.id = pharmacy_fills.prescription_id
group by prescriptions.drug_name
""")

In [96]:
for row in mycursor.fetchall():
    print(row)

('Avafoxin', Decimal('1.7500'))
('Cleotrana', Decimal('1.3333'))
('Primalovir', Decimal('2.0000'))
('Glucozepam Amcipentin', Decimal('1.5000'))
('Olanzanafine', Decimal('2.0000'))
('Abnazole Toleluble', Decimal('4.0000'))
('Dantopex Quixilinum', Decimal('2.0000'))
('Quixiposide', Decimal('2.0000'))
('Kanulin', Decimal('0.0000'))


In [97]:
# 7.For each pharmacy, find all the drugs that were prescribed to a patient and never filled at that pharmacy.

In [98]:
mycursor.execute("""
select pharmacies.name, prescriptions.drug_name
from pharmacies cross join prescriptions
where not exists (select pharmacies.name, prescriptions.drug_name
                  from pharmacy_fills
                  where pharmacy_fills.prescription_id = prescriptions.id and pharmacy_fills.pharmacy_id = pharmacies.id)
order by pharmacies.name, prescriptions.drug_name
""")

In [99]:
for row in mycursor.fetchall():
    print(row)

(' Bayside Pharmacy', 'Abnazole Toleluble')
(' Bayside Pharmacy', 'Avafoxin')
(' Bayside Pharmacy', 'Avafoxin')
(' Bayside Pharmacy', 'Avafoxin')
(' Bayside Pharmacy', 'Avafoxin')
(' Bayside Pharmacy', 'Cleotrana')
(' Bayside Pharmacy', 'Cleotrana')
(' Bayside Pharmacy', 'Cleotrana')
(' Bayside Pharmacy', 'Dantopex Quixilinum')
(' Bayside Pharmacy', 'Glucozepam Amcipentin')
(' Bayside Pharmacy', 'Kanulin')
(' Bayside Pharmacy', 'Olanzanafine')
(' Bayside Pharmacy', 'Primalovir')
(' Bayside Pharmacy', 'Primalovir')
(' Bayside Pharmacy', 'Quixiposide')
(' Beantown Drugs', 'Abnazole Toleluble')
(' Beantown Drugs', 'Avafoxin')
(' Beantown Drugs', 'Avafoxin')
(' Beantown Drugs', 'Avafoxin')
(' Beantown Drugs', 'Avafoxin')
(' Beantown Drugs', 'Cleotrana')
(' Beantown Drugs', 'Cleotrana')
(' Beantown Drugs', 'Dantopex Quixilinum')
(' Beantown Drugs', 'Glucozepam Amcipentin')
(' Beantown Drugs', 'Glucozepam Amcipentin')
(' Beantown Drugs', 'Kanulin')
(' Beantown Drugs', 'Olanzanafine')
(' Bean